In [1]:
import requests
from urllib.parse import quote

client_id = "SWe77FM8c_EjAm5Y93h5"
client_secret = "mAz_OvLf7T"

def naver_shop_search(query, display=10):
    encoded_query = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded_query}&display={display}&start=1&sort=sim&exclude=used:cbshop:rental"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code}

result = naver_shop_search("영양제", 10)

if 'items' in result:
  for item in result['items']:
    if item['brand'] != '':
      print(f"브랜드: {item['brand']}")
      print(f"이름: {item['title']}")
      print(f"상품ID: {item['productId']}")
      print(f"링크: {item['link']}")
      print("-" * 50)
else:
  print("Error:", result.get('error', 'Unknown error'))


브랜드: 오쏘몰
이름: 오쏘몰 이뮨 멀티비타민 미네랄 (액상 20ml + 정제 919mg) x 30개입, 1개
상품ID: 51929317174
링크: https://search.shopping.naver.com/catalog/51929317174
--------------------------------------------------
브랜드: 센트룸
이름: 센트룸 실버 포 우먼 종합 멀티 비타민 <b>영양제</b> 112정 코스트코 비타민B12 대용량
상품ID: 86787803460
링크: https://smartstore.naver.com/main/products/9243303137
--------------------------------------------------
브랜드: 조아제약
이름: 잘크톤 어린이비타민 멀티 젤리 아이 어린이<b>영양제</b> 키즈비타민 아기 유아 튼튼
상품ID: 85645949140
링크: https://smartstore.naver.com/main/products/8101448817
--------------------------------------------------
브랜드: 네츄럴라이프얼라이브
이름: 얼라이브 원스데일리 멀티/포맨/ 우먼 60정 비타민B군 미네랄 종합<b>영양제</b>
상품ID: 10095168405
링크: https://smartstore.naver.com/main/products/309612084
--------------------------------------------------
브랜드: 센트룸
이름: 센트룸 실버 포 맨 종합 멀티 비타민 112정 코스트코 남성 50+ 대용량 <b>영양제</b>
상품ID: 86788593520
링크: https://smartstore.naver.com/main/products/9244093197
--------------------------------------------------
브랜드: 종근당건강
이름: [1+1]종근당 아임비타 이뮨플러스 샷 2박스

In [2]:
!pip install -q chromadb sentence-transformers  # Chroma DB와 임베딩 모델 설치

import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.6 MB/s eta 0:

In [ ]:
# 1. 데이터 전처리 함수
def preprocess_items(items):
    processed = []
    for item in items:
        data = {
            "id": item['productId'],
            "text": f"{item['title']} {item['brand']} {item['maker']} {item['category3']}",  # 임베딩용 텍스트
            "metadata": {
                "price": int(item['lprice']),
                "brand": item['brand'],
                "category": f"{item['category1']}>{item['category2']}>{item['category3']}",
                "link": item['link']
            }
        }
        processed.append(data)
    return processed

# 2. Chroma DB 설정

# Chroma DB 경로 초기화
# !rm -rf ./naver_shopping_db  # Colab에서 실행시

# 클라이언트 재생성
client = chromadb.PersistentClient(path="./naver_shopping_db")
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)
# 기존 컬렉션이 있으면 로드, 없으면 새로 생성
# collection = client.get_or_create_collection(
#     name="products",
#     embedding_function=embedding_model  # 반드시 임베딩 함수 재지정
# )
# 기존 컬렉션 삭제 후 재생성
try:
    client.delete_collection("products")
except:
    pass
collection = client.create_collection(
    name="products",
    embedding_function=embedding_model,  # 반드시 임베딩 함수 재지정
    metadata={"hnsw:space": "cosine"}  # 거리 측정 방식 변경
)  # 원래 코드 유지

# 3. 데이터 삽입 (API 결과 사용)
if 'items' in result:
    processed_data = preprocess_items(result['items'])

    # 벡터 DB에 문서 추가
    collection.add(
        ids=[item['id'] for item in processed_data],
        documents=[item['text'] for item in processed_data],
        metadatas=[item['metadata'] for item in processed_data]
    )
    print("✅ 성공적으로", len(processed_data), "개 상품 저장됨")
else:
    print("Error:", result.get('error', '데이터 없음'))

# 4. 검색 예시 함수
def search_products(query, n_results=3):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    for idx, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
        print(f"🔍 [{idx}] {doc}")
        print(f"   가격: {meta['price']:,}원 | 브랜드: {meta['brand']}")
        print(f"   카테고리: {meta['category']}")
        print(f"   링크: {meta['link']}")
        print("-"*50)

In [4]:
search_products("임산부에게 좋은 영양제", 5)

🔍 [1] 잘크톤 어린이비타민 멀티 젤리 아이 어린이<b>영양제</b> 키즈비타민 아기 유아 튼튼 조아제약  비타민제
   가격: 31,100원 | 브랜드: 조아제약
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/8101448817
--------------------------------------------------
🔍 [2] 올인원<b>영양제</b> 이뮨 멀티팩 남성 여성 종합비타민   비타민제
   가격: 38,800원 | 브랜드: 
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/9057792910
--------------------------------------------------
🔍 [3] 얼라이브 원스데일리 멀티/포맨/ 우먼 60정 비타민B군 미네랄 종합<b>영양제</b> 네츄럴라이프얼라이브 네이쳐스웨이 비타민제
   가격: 23,900원 | 브랜드: 네츄럴라이프얼라이브
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/309612084
--------------------------------------------------
🔍 [4] [1+1]종근당 아임비타 이뮨플러스 샷 2박스 올인원<b>영양제</b> 마시는 액상 비타민 종근당건강  비타민제
   가격: 23,500원 | 브랜드: 종근당건강
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/8566190007
--------------------------------------------------
🔍 [5] 센트룸 실버 포 맨 종합 멀티 비타민 112정 코스트코 남성 50+ 대용량 <b>영양제</b> 센트룸 화이자 비타민제
   가격: 34,900원 | 브랜드: 센트

In [5]:
search_products("어르신들을 위한 영양제", 5)

🔍 [1] 얼라이브 원스데일리 멀티/포맨/ 우먼 60정 비타민B군 미네랄 종합<b>영양제</b> 네츄럴라이프얼라이브 네이쳐스웨이 비타민제
   가격: 23,900원 | 브랜드: 네츄럴라이프얼라이브
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/309612084
--------------------------------------------------
🔍 [2] 센트룸 실버 포 맨 종합 멀티 비타민 112정 코스트코 남성 50+ 대용량 <b>영양제</b> 센트룸 화이자 비타민제
   가격: 34,900원 | 브랜드: 센트룸
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/9244093197
--------------------------------------------------
🔍 [3] 올인원<b>영양제</b> 이뮨 멀티팩 남성 여성 종합비타민   비타민제
   가격: 38,800원 | 브랜드: 
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/9057792910
--------------------------------------------------
🔍 [4] [1+1]종근당 아임비타 이뮨플러스 샷 2박스 올인원<b>영양제</b> 마시는 액상 비타민 종근당건강  비타민제
   가격: 23,500원 | 브랜드: 종근당건강
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/8566190007
--------------------------------------------------
🔍 [5] 센트룸 포 맨 더블업 종합 멀티 비타민 112정 코스트코 남성 대용량 <b>영양제</b> 센트룸 콜마비앤에이치 비타민제
   가격: 34,900원 | 브

In [ ]:
# Hugging Face 및 Google API 인증
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN') # Hugging Face 액세스 토큰
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Google Custom Search API 키
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID') # Programmable Search Engine ID
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # OpenAI API Key

In [ ]:
# 2. Google Programmable Search Engine 설정
from googleapiclient.discovery import build

def google_search(query, num=3):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]

# Gemma-3-1b-it 모델 초기화
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it", token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)



In [9]:
!pip install -q langchain langchain-community langchain-core chromadb langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 22.0 MB/s eta 0:00:00


In [24]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

# 2. 검색 도구 정의 (Function Calling)
@tool
def search_products(query: str, max_results: int = 3) -> list:
    """제품 검색을 위해 Chroma DB를 쿼리하는 도구"""
    results = collection.query(
        query_texts=[query],
        n_results=max_results
    )
    return [{
        "제품": doc,
        "가격": meta['price'],
        "링크": meta['link']
    } for doc, meta in zip(results['documents'][0], results['metadatas'][0])]

# 3. 에이전트 설정
# llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.3, openai_api_key=OPENAI_API_KEY)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3, openai_api_key=OPENAI_API_KEY)
llm_with_tools = llm.bind_tools([search_products])

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영양제 전문 약사입니다. 검색 도구를 적절히 사용해 질문에 답변하세요."),
    MessagesPlaceholder("chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

tools = [search_products]
agent = create_openai_tools_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
# 4. 실행 예시
response = agent_executor.invoke({
    "input": "임산부를 위한 영양제를 추천해줘"
})
print(response['output'])